# Vehicle Sales Price Predictions Workshop - Part 3 of 3


## Inference Pipeline

In order to make a machine learning system from this dataset, we have structured the service into 3 pipelines:

1. feature engineering pipeline notebook (see Part 1)
2. training pipeline notebook (see Part 2)
3. inferencing pipeline notebook (this Part 3)

This notebook will outline the third step, ie. the inference pipeline.

In [1]:
# We need to install a library to deploy the model. This install throws an error in colab, but will still work.
!pip install --quiet "hsfs[python] @ git+https://github.com/logicalclocks/feature-store-api@master#subdirectory=python"

In [2]:
# We will use the Hopsorks Model Registry to instantiate the Model

import hopsworks
import joblib
import torch
import torch.nn as nn

proj = hopsworks.login()
fs = proj.get_feature_store()
mr = proj.get_model_registry()

feature_view = fs.get_feature_view("car_prices", version=1)

model = mr.get_model(
    "car_prices",
    version=1,
)

# Download the model directory from the Model Registry
model_dir = model.download()

# Load the model using joblib from the downloaded model directory
label_encoders = joblib.load(model_dir + "/label_encoders.pkl")

# Definition of the model
class DeepRegressor(nn.Module):
    def __init__(self, input_size):
        super(DeepRegressor, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load the file of the model
model_data = torch.load(model_dir + '/regression_model.pth', map_location=torch.device('cpu'))

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/14457
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# Check the type of the loaded object
if isinstance(model_data, dict) and 'model_state_dict' in model_data:
    state_dict = model_data['model_state_dict']
elif isinstance(model_data, torch.nn.Module):
    state_dict = model_data.state_dict()
else:
    raise ValueError("The file does not contain a valid PyTorch model or the format is unexpected.")

# Show weights of each layer
print("Show the weights of the different layers of the model:")
for layer_name, weights in state_dict.items():
    print(f"\nLayer: {layer_name}")
    print(f"Weights: {weights.shape}")
    print(weights)

# If the object contains another structure, inspect it
if isinstance(model_data, dict):
    print("\nComplete structure of the saved model:")
    for key, value in model_data.items():
        if key != 'model_state_dict':
            print(f"\nStructure of the key '{key}':")
            if isinstance(value, torch.Tensor):
                print(f"  - Tensor with shape : {value.shape}")
            elif isinstance(value, dict):
                print("  - Dictionnary with keys :")
                for subkey in value.keys():
                    print(f"    - {subkey}")
            else:
                print(f"  - Type : {type(value)}")


# If the object is directly the model, display its architecture
elif isinstance(model_data, torch.nn.Module):
    print("\nThe saved model is a direct PyTorch model.")
    print("Architecture of the model :")
    print(model_data)

Show the weights of the different layers of the model:

Layer: fc1.weight
Weights: torch.Size([64, 10])
tensor([[-1.1307e-01, -2.8962e-01, -6.0175e-01, -5.0341e-01, -4.7569e-01,
         -3.0303e-01, -2.0566e-01, -2.6545e-03,  5.8319e-02, -4.6308e-01],
        [ 3.6571e-01, -2.7286e-01,  9.2609e-03, -9.4895e-01, -3.9846e-01,
          1.6230e-02,  4.9595e-01, -3.8214e-01, -1.5098e-01,  1.8051e-01],
        [-1.2615e-01, -1.6503e-01,  1.1018e-01, -3.5331e-01, -5.8650e-01,
         -1.9680e-01, -5.6965e-01,  1.4361e-04, -2.3954e-01, -2.1430e-01],
        [ 5.9170e-03, -1.6541e-01, -4.7092e-01, -3.1342e-01, -1.5425e-01,
         -2.7775e-02,  5.4793e-01, -3.1691e-01, -2.9763e-01,  1.1253e-01],
        [ 3.8434e-01,  3.3014e-02, -1.0500e+00, -7.2068e-01, -7.1965e-02,
         -7.0629e-01,  2.6572e-01, -6.9269e-02, -2.2294e-01, -5.7815e-01],
        [-8.7996e-01, -6.1422e-01, -2.0522e-01, -5.0140e-02, -7.5284e-01,
          2.7040e-02, -1.0937e+00, -5.4314e-03, -8.0150e-01, -5.0534e-01],
  

In [4]:
test_data = feature_view.get_batch_data(start_time="2015-07-01 00:00")
test_data

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.89s) 


,year,make,model,trim,body,transmission,condition,odometer,color,interior
0,2004,Nissan,Murano,SE,suv,automatic,21.0,135851.0,white,beige
1,2012,Honda,Odyssey,Touring,minivan,automatic,37.0,92904.0,black,gray
2,2014,Toyota,Corolla,LE,sedan,automatic,2.0,26987.0,red,gray
3,2012,Lexus,RX 350,Base,suv,automatic,35.0,30235.0,white,beige
4,2012,Ford,Flex,SEL,Wagon,automatic,39.0,28320.0,red,black
...,...,...,...,...,...,...,...,...,...,...
829,2011,Chevrolet,Equinox,LT1,suv,automatic,19.0,56996.0,black,black
830,2014,Toyota,Sienna,L 7-Passenger,minivan,automatic,31.0,4928.0,gray,gray
831,2012,Hyundai,Sonata,GLS,sedan,automatic,44.0,38668.0,blue,gray
832,2013,Nissan,Altima,2.5,sedan,automatic,33.0,37159.0,black,black


In [5]:
from sklearn.preprocessing import LabelEncoder

# Define the function to encode categorical data
def encode_categorical_data(dataset, label_encoders):    
    # Iterate over the columns of the DataFrame
    for column in dataset.columns:
        # Check if the column is of type 'object' (categorical)
        if dataset[column].dtype == 'object':
            # Retrieve already fitted LabelEncoder
            label_encoder = label_encoders[column.lower()]
            
            # Perform encoding on unique column values
            dataset[column] = label_encoder.transform(dataset[column])
    return dataset

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd

# Define the function to predict selling prices
def predict_selling_price(data, label_encoders, model):
        
    # Preprocess the data
    processed_data = encode_categorical_data(data, label_encoders)
    
    # Convert data to PyTorch tensors
    X_test = torch.tensor(processed_data.values.astype(np.float32))
    
    # Pass data to model for prediction
    with torch.no_grad():
        model.eval()
        predictions = model(X_test).numpy()
        
    predictions_df = pd.DataFrame(predictions, columns=['predicted_sale_price'])
    data['predicted_sale_price'] = predictions_df['predicted_sale_price']
    return data

df_encoded = predict_selling_price(test_data, label_encoders, model_data)
df_encoded

,year,make,model,trim,body,transmission,condition,odometer,color,interior,predicted_sale_price
0,2004,33,404,853,72,0,21.0,135851.0,17,0,9138.729492
1,2012,16,417,1019,62,0,37.0,92904.0,1,6,9468.860352
2,2014,45,149,634,69,0,2.0,26987.0,14,6,9897.079102
3,2012,24,471,321,72,0,35.0,30235.0,17,0,9994.019531
4,2012,12,237,875,38,0,39.0,28320.0,14,1,10007.195312
...,...,...,...,...,...,...,...,...,...,...,...
829,2011,7,202,661,72,0,19.0,56996.0,1,1,9653.800781
830,2014,45,535,628,62,0,31.0,4928.0,7,6,10152.079102
831,2012,17,556,547,69,0,44.0,38668.0,2,6,9977.411133
832,2013,33,52,81,69,0,33.0,37159.0,1,1,9857.138672


### 8. Use your trained model to make price estimates

Make a prediction function to load the model trained and saved in the model_regressor.pth file and then test on data that the user will enter manually. As a reminder, the categorical data was encoded and saved in a label_encoders.pth file. There are also numeric variables that the user must indicate.

In [7]:
!pip install --quiet gradio==3.48.0

In [8]:
import gradio as gr
from functools import partial
import warnings
warnings.filterwarnings('ignore')

# Define the method to print out the values entered
def print_values(year, make, model, trim, body, transmission, condition, odometer, color, interior, model_data, label_encoders):
    data = {
        "Year": [year],
        "Make": [make],
        "Model": [model],
        "Trim": [trim],
        "Body": [body],
        "Transmission": [transmission],
        "Condition": [condition],
        "Odometer": [odometer],
        "Color": [color],
        "Interior": [interior]
    }
    df = pd.DataFrame(data)
        
    df_encoded = predict_selling_price(df, label_encoders, model_data)
    return df_encoded.iloc[0]['predicted_sale_price']

print_values_partial = partial(
    print_values, 
    model_data=model_data, 
    label_encoders=label_encoders,
)

In [9]:
# Create the Gradio interface
with gr.Blocks() as demo:
    with gr.Row():
        year = gr.Number(label="Year", value=2014)
        make = gr.Dropdown(label="Make", choices=["Toyota", "Ford", "Volkswagen", "BMW"], value="Toyota")
        model = gr.Dropdown(label="Model", choices=["Prius", "Mustang", "Jetta", "X1"], value="Prius")
        trim = gr.Dropdown(label="Trim", choices=["Base", "SL", "GLS", "Luxury"], value="Base")
        body = gr.Dropdown(label="Body", choices=["Hatchback", "Sedan", "SUV", "Minivan"], value="Hatchback")

    with gr.Row():
        transmission = gr.Dropdown(label="Transmission", choices=["automatic", "manual"], value="automatic")
        condition = gr.Number(label="Condition", value=45.0)
        odometer = gr.Number(label="Odometer", value=33761.0)
        color = gr.Dropdown(label="Color", choices=["red", "white", "black", "silver"], value="red")
        interior = gr.Dropdown(label="Interior", choices=["black", "gray", "brown"], value="black")

    submit_button = gr.Button("Submit")
    output = gr.JSON(label="Entered Values")

    submit_button.click(
        print_values_partial,
        inputs=[year, make, model, trim, body, transmission, condition, odometer, color, interior],
        outputs=output,
    )

# Launch the interface
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://4cb53a497e1050d23d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


This completes the entire process of feature engineering, training and inferencing pipelines, and therefore the delivery of an end-to-end machine learning system.

---